# 가중 임베딩 (weighted embedding)

## Reference
* ratsgo님의 가중 임베딩 함수
* a simple but tough to beat baseline for sentence embeddings, Sanjeev Arora et al.

### 수정 사항
* 인코딩 에러 문제로, 모든 with open()에서 encoding='utf-8' 옵션을 추가함.

### 추가하고 싶은 사항 (아직 추가 안함)
* ~pred_by_batch의 output을 모델이 예측한 라벨링, 그리고 이것과 실제 라벨링의 confusion matric을 반환하도록.~ (완료)

### 함수의 인자 설명
* train_fname: tokenized가 완료된 문장과 라벨이 ␞로 한줄에 이어져 있는 파일의 경로. 여기에서는 네이버 평점 데이터만 해당.
* embedding_fname: word2vec 모델 경로
* model_fname: 모델을 저장할 경로
* embedding_corpus_fname: word2vec을 만들 때, 사용한 전체 corpus. 한줄에 tokenized가 완료된 문장 하나가 옴. 여기서는 네이버 평점, korquad, ko_wiki 코퍼스 모두 포함.
* embedding_method: 임베딩 방법
* is_weighted: 가중 임베딩을 사용할건지, 그냥 단순 평균 임베딩을 사용할건지. 두 가지 결과를 비교해볼 수 있다.
* dim: 기본은 100차원. 임베딩 벡터의 차원 수이다.
* tokenizer_name: 어떤 tokenizer을 사용할 것인지. 여기서 기본 tokenizer는 mecab이다.

In [1]:
from weighted_embedding import CBoWModel

In [2]:
train_fname = 'processed_ratings_train.txt'
embedding_fname = 'word2vec'
model_fname = 'weighted_word2vec_result'
embedding_corpus_fname = 'corpus_mecab.txt'

In [3]:
cbow = CBoWModel(train_fname, embedding_fname, model_fname,
                embedding_corpus_fname, embedding_method='word2vec')

loading weighted embeddings, complete!
load Continuous Bag of Words model


In [4]:
# 가중 임베딩 모델 input 문장에 대한 예시
print('이 영화 꿀잼: ', cbow.predict('이 영화 꿀잼'))
print('절대 안봄: ', cbow.predict('절대 안봄'))

이 영화 꿀잼:  1
절대 안봄:  0


* 테스트 데이터 부르고 모델링 결과 맞춰보기
* 테스트 데이터 5만개 모두 쓰면 memory error가 떠서 10000개만 사용한다.

In [5]:
# load test data and evaluate
tokenized_sentences, test_labels = [], []
a = 0
with open('processed_ratings_test.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        a+=1
        sent_split = line.strip().split("\u241E")
        tokenized_sentences.append(sent_split[0].split(' '))
        test_labels.append(sent_split[1])
        
        if a == 1000: break

In [7]:
pred_labels = cbow.predict_by_batch(tokenized_sentences, test_labels)

In [8]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pred_labels, test_labels)

array([[343, 267],
       [149, 241]], dtype=int64)

지금은 1000개만 evaluate해서 정확도가 다소 떨어지는 것으로 보인다.